# STZ Stress Relaxation — Aging in Colloidal Glasses

**Shear Transformation Zone model — Stress relaxation with aging**

## Learning Objectives

- Understand stress relaxation in STZ: gamma_dot=0, stress decays through plastic rearrangements
- Fit relaxation data from laponite colloidal glass at multiple aging times
- Track chi_inf(t_wait) to observe physical aging (deepening energy traps)
- Compare STZ relaxation physics with SGR power-law decay

## Prerequisites

- Notebook 01 (STZ flow curve basics)
- Understanding of stress relaxation G(t)

## Estimated Runtime

- Fast demo (1 chain): ~3-5 min
- Full run (4 chains): ~8-15 min

## 1. Setup

In [ ]:
# Colab setup
import sys

IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
    %pip install -q rheojax
    import os
    os.environ["JAX_ENABLE_X64"] = "true"
    print("RheoJAX installed successfully.")

In [ ]:
%matplotlib inline
import os
import time
import warnings

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.stz import STZConventional

jax, jnp = safe_import_jax()
verify_float64()

warnings.filterwarnings("ignore", category=FutureWarning)
print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")

## 2. Theory: Relaxation in STZ

After a step strain $\gamma_0$, the applied shear rate is zero ($\dot{\gamma} = 0$). Stress decays as stored elastic energy is dissipated through plastic STZ rearrangements:

$$\frac{d\sigma}{dt} = -G_0 \, \dot{\gamma}_{\text{pl}}(\sigma, \chi, \Lambda)$$

The plastic rate depends on:
- **$\chi(t)$** evolving toward $\chi_{\infty}$ (or cooling if $\chi > \chi_{\infty}$)
- **$\Lambda(t)$** tracking $\exp(-e_z/\chi)$ on timescale $\tau_\beta$

### Initial conditions for relaxation

The model starts with $\chi = \chi_{\infty}$ and $\sigma_0 = \sigma_y$ (default), representing a sample that was previously sheared to steady state.

### Aging interpretation

In colloidal glasses, the **waiting time** $t_{\text{wait}}$ after preparation controls the depth of energy traps. Longer waiting = deeper traps = lower $\chi_{\infty}$ = slower relaxation. Fitting STZ to multiple aging times reveals how $\chi_{\infty}(t_{\text{wait}})$ decreases — a direct measure of physical aging.

### Material-Model Compatibility

**Laponite clay** is a synthetic colloidal glass that ages continuously after preparation. The platelets form a disordered arrested structure, making it a genuine STZ target material. The aging-dependent relaxation modulus G(t) directly probes the evolving energy landscape.

**Data conversion:** The raw data gives G(t) = relaxation modulus. We convert to stress via $\sigma(t) = G(t) \cdot \gamma_0$ with an assumed step strain $\gamma_0 = 0.01$ (1%). The choice of $\gamma_0$ affects the absolute scale of $\sigma_y$ and $G_0$ but not $\chi_{\infty}$, $\tau_0$, or $e_z$.

## 3. Load Data

In [ ]:
# Robust path resolution for execution from any directory
import sys
from pathlib import Path
_nb_dir = Path(__file__).parent if "__file__" in dir() else Path.cwd()
_stz_candidates = [_nb_dir, Path("examples/stz"), _nb_dir.parent / "stz"]
for _p in _stz_candidates:
    if (_p / "stz_tutorial_utils.py").exists():
        sys.path.insert(0, str(_p.resolve()))
        break

from stz_tutorial_utils import load_laponite_relaxation

aging_times = [600, 1200, 1800, 2400, 3600]
datasets = {}

for t_age in aging_times:
    t_data, G_t = load_laponite_relaxation(t_age=t_age)
    datasets[t_age] = (t_data, G_t)
    print(f"t_age = {t_age:5d} s: {len(t_data)} points, G range [{G_t.min():.0f}, {G_t.max():.0f}] Pa")

In [ ]:
fig, ax = plt.subplots(figsize=(9, 6))
colors = plt.cm.viridis(np.linspace(0.2, 0.9, len(aging_times)))

for i, t_age in enumerate(aging_times):
    t_data, G_t = datasets[t_age]
    ax.loglog(t_data, G_t, "o", markersize=4, color=colors[i],
              label=f"$t_{{wait}}$ = {t_age} s")

ax.set_xlabel("Time [s]")
ax.set_ylabel("G(t) [Pa]")
ax.set_title("Laponite Clay Relaxation — 5 Aging Times")
ax.legend()
ax.grid(True, alpha=0.3, which="both")
plt.tight_layout()
display(fig)
plt.close(fig)

The relaxation modulus increases and slows with aging time — hallmark of physical aging in colloidal glasses.

## 4. NLSQ Fitting (Single Aging Time)

We first fit the longest aging time ($t_{\text{wait}} = 3600$ s), then sweep all five.

In [ ]:
from stz_tutorial_utils import compute_fit_quality

# Convert G(t) to stress: sigma = G(t) * gamma_0
gamma_0 = 0.01  # Assumed step strain (1%)

t_data, G_t = datasets[3600]
stress_data = G_t * gamma_0

model = STZConventional(variant="standard")

# Set bounds BEFORE values — use set_bounds() to update both bounds and constraints
model.parameters.set_bounds("G0", (10.0, 5000.0))
model.parameters["G0"].value = 300.0
model.parameters.set_bounds("sigma_y", (0.1, 50.0))
model.parameters["sigma_y"].value = 3.0
model.parameters.set_bounds("chi_inf", (0.02, 0.5))
model.parameters["chi_inf"].value = 0.1
model.parameters.set_bounds("tau0", (1e-8, 1e0))
model.parameters["tau0"].value = 1e-4
model.parameters.set_bounds("epsilon0", (0.01, 1.0))
model.parameters["epsilon0"].value = 0.1
model.parameters.set_bounds("c0", (0.1, 50.0))
model.parameters["c0"].value = 1.0
model.parameters.set_bounds("ez", (0.1, 5.0))
model.parameters["ez"].value = 1.0
model.parameters.set_bounds("tau_beta", (0.01, 100.0))
model.parameters["tau_beta"].value = 1.0

# Initial stress for relaxation
sigma_0 = float(stress_data[0])

t0 = time.time()
model.fit(
    t_data, stress_data,
    test_mode="relaxation",
    sigma_0=sigma_0, method='scipy')
t_nlsq = time.time() - t0

stress_at_data = model.predict(t_data)
quality = compute_fit_quality(stress_data, stress_at_data)

print(f"NLSQ fit time: {t_nlsq:.2f} s")
print(f"R-squared: {quality['r_squared']:.6f}")
print("\nFitted parameters:")
relax_params = ["G0", "sigma_y", "chi_inf", "tau0", "epsilon0", "c0", "ez", "tau_beta"]
for name in relax_params:
    val = model.parameters.get_value(name)
    print(f"  {name:10s} = {val:.4g}")

In [ ]:
t_fine = np.logspace(np.log10(t_data.min()), np.log10(t_data.max()), 200)
stress_pred = model.predict(t_fine)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Left: fit
ax1.loglog(t_data, stress_data, "ko", markersize=5, label="Data ($t_{wait}$=3600s)")
ax1.loglog(t_fine, stress_pred, "-", lw=2, color="C0", label="STZ fit")
ax1.set_xlabel("Time [s]")
ax1.set_ylabel("Stress [Pa]")
ax1.set_title("Relaxation Fit ($t_{wait}$ = 3600 s)")
ax1.legend()
ax1.grid(True, alpha=0.3, which="both")

# Right: residuals
stress_at_data = model.predict(t_data)
res = (stress_data - stress_at_data) / stress_data * 100
ax2.semilogx(t_data, res, "o-", markersize=4, color="C0")
ax2.axhline(0, color="black", linestyle="--", alpha=0.5)
ax2.set_xlabel("Time [s]")
ax2.set_ylabel("Relative residual [%]")
ax2.set_title("Residual Analysis")
ax2.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

## 5. Aging Sweep: chi_inf vs t_wait

Fit all 5 aging times and track how the effective temperature evolves.

In [ ]:
from stz_tutorial_utils import compute_fit_quality

aging_results = {}

for t_age in aging_times:
    t_data_i, G_t_i = datasets[t_age]
    stress_i = G_t_i * gamma_0
    sigma_0_i = float(stress_i[0])

    m = STZConventional(variant="standard")

    # Set bounds BEFORE values — use set_bounds() to update constraints
    m.parameters.set_bounds("G0", (10.0, 5000.0))
    m.parameters["G0"].value = 300.0
    m.parameters.set_bounds("sigma_y", (0.1, 50.0))
    m.parameters["sigma_y"].value = 3.0
    m.parameters.set_bounds("chi_inf", (0.02, 0.5))
    m.parameters["chi_inf"].value = 0.1
    m.parameters.set_bounds("tau0", (1e-8, 1e0))
    m.parameters["tau0"].value = 1e-4
    m.parameters.set_bounds("epsilon0", (0.01, 1.0))
    m.parameters["epsilon0"].value = 0.1
    m.parameters.set_bounds("c0", (0.1, 50.0))
    m.parameters["c0"].value = 1.0
    m.parameters.set_bounds("ez", (0.1, 5.0))
    m.parameters["ez"].value = 1.0
    m.parameters.set_bounds("tau_beta", (0.01, 100.0))
    m.parameters["tau_beta"].value = 1.0

    try:
        m.fit(t_data_i, stress_i, test_mode="relaxation", sigma_0=sigma_0_i, method='scipy')
        stress_pred_i = m.predict(t_data_i)
        q = compute_fit_quality(stress_i, stress_pred_i)
        aging_results[t_age] = {
            "chi_inf": m.parameters.get_value("chi_inf"),
            "sigma_y": m.parameters.get_value("sigma_y"),
            "tau0": m.parameters.get_value("tau0"),
            "r_squared": q["r_squared"],
        }
        print(f"t_age={t_age:5d}s: chi_inf={aging_results[t_age]['chi_inf']:.4f}, "
              f"R^2={aging_results[t_age]['r_squared']:.4f}")
    except Exception as e:
        print(f"t_age={t_age:5d}s: FAILED ({e})")

In [ ]:
t_ages_fit = sorted(aging_results.keys())
chi_vals = [aging_results[t]["chi_inf"] for t in t_ages_fit]

fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(t_ages_fit, chi_vals, "o-", markersize=8, lw=2, color="C2")
ax.set_xlabel("Waiting time $t_{wait}$ [s]")
ax.set_ylabel("$\\chi_{\\infty}$ (effective temperature)")
ax.set_title("Physical Aging: $\\chi_{\\infty}$ Decreases with $t_{wait}$")
ax.grid(True, alpha=0.3)
plt.tight_layout()
display(fig)
plt.close(fig)

**Key insight:** Decreasing $\chi_{\infty}$ with $t_{\text{wait}}$ means the system is falling into deeper energy minima — fewer STZs are available, and plastic rearrangements become progressively harder. This is the STZ interpretation of physical aging.

## 6. Bayesian Inference (Single Aging Time)

In [ ]:
initial_values = {
    name: model.parameters.get_value(name)
    for name in model.parameters.keys()
}

# --- Fast demo config ---
NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1
# NUM_WARMUP = 1000; NUM_SAMPLES = 2000; NUM_CHAINS = 4  # production

# Use t_age=3600 data
t_data, G_t = datasets[3600]
stress_data = G_t * gamma_0

t0 = time.time()
result = model.fit_bayesian(
    t_data,
    stress_data,
    test_mode="relaxation",
    num_warmup=NUM_WARMUP,
    num_samples=NUM_SAMPLES,
    num_chains=NUM_CHAINS,
    initial_values=initial_values,
    seed=42,
)
t_bayes = time.time() - t0
print(f"Bayesian inference time: {t_bayes:.1f} s")

In [ ]:
from stz_tutorial_utils import print_convergence_summary, print_parameter_comparison

print_convergence_summary(result, relax_params)

In [ ]:
idata = result.to_inference_data()

axes = az.plot_trace(idata, var_names=relax_params, figsize=(12, 16))
fig = axes.ravel()[0].figure
fig.suptitle("Trace Plots (Relaxation)", fontsize=14, y=1.02)
plt.tight_layout()
display(fig)
plt.close(fig)

In [ ]:
axes = az.plot_forest(
    idata,
    var_names=relax_params,
    combined=True,
    hdi_prob=0.95,
    figsize=(10, 6),
)
fig = axes.ravel()[0].figure
plt.tight_layout()
display(fig)
plt.close(fig)

In [ ]:
posterior = result.posterior_samples
print_parameter_comparison(model, posterior, relax_params)

## 7. Cross-Model Note: STZ vs SGR Relaxation

Both STZ and SGR can fit this laponite data, but with different physics:

| Feature | STZ | SGR |
|---------|-----|-----|
| Relaxation mechanism | Activated plastic rearrangements | Trap-hopping in energy landscape |
| Aging parameter | $\chi_{\infty}(t_{\text{wait}})$ decreases | $x(t_{\text{wait}})$ decreases |
| Functional form | ODE-based (exponential-like decay) | Power-law $G(t) \sim t^{x-2}$ |
| Parameters | 8 (ODE system) | 3 ($x$, $G_0$, $\tau_0$) |

See **SGR Notebook 02** for the SGR perspective on the same laponite data.

## 8. Save Results

In [ ]:
from stz_tutorial_utils import save_stz_results

output_dir = os.path.join("..", "outputs", "stz", "relaxation")
save_stz_results(model, result, output_dir, "relaxation")

## Key Takeaways

1. **STZ relaxation = stress decay via plastic rearrangements** — even at gamma_dot=0, activated STZ events dissipate stored elastic energy
2. **chi_inf(t_wait) tracks physical aging** — deeper traps mean lower effective temperature and slower relaxation
3. **8 parameters from relaxation** — transient data activates G0, epsilon0, c0, tau_beta that are invisible to steady-state
4. **STZ vs SGR** — same data, different physics: activated events vs trap-hopping
5. **Laponite is a genuine STZ target** — colloidal glass with arrested disordered structure

## Next Steps

- **Notebook 04**: Creep with yield stress bifurcation
- **Notebook 02**: Startup shear with stress overshoot